In [1]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [2]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [3]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [4]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [5]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = ' lab.aimet.tech:9092'

In [6]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [7]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [8]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [9]:
vid = 'V763258'
token = '144c0db0459d005a4beef900f0c0c531'
tx_data = next(txconsumer).value
txid, payer, payee, amount = tx_data['txid'], tx_data['payer'], tx_data['payee'], tx_data['amount']

signature = gen_signature(txid, payer, payee, amount, token)
print('txid: ', txid)
print('---')

verification_message = {
    'vid': vid,
    'txid': txid,
    'signature': signature
}

serialized_message = serialize(submitschema, verification_message)
producer.send('submit', serialized_message)

result = next(resultconsumer)
if result.value['code'] == 200 and result.value['vid'] == vid and result.value['txid'] == txid:
    print('Verified')

print(f'{"timestamp:":12}{result.value["timestamp"]}')
print(f'{"vid:":12}{result.value["vid"]}')
print(f'{"txid:":12}{result.value["txid"]}')
print(f'{"checksum:":12}{result.value["checksum"]}')
print(f'{"code:":12}{result.value["code"]}')

txid:  TX09568
---
Verified
timestamp:  1711324114
vid:        V763258
txid:       TX09568
checksum:   a7f98af9a8724eca17d1fbfbb44efff8
code:       200
